<a href="https://colab.research.google.com/github/ziyasarican/PythonMachineLearning/blob/main/MultipleLinearRegressionEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
# amaç humidty'yi tahmin etmek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("odev_tenis.csv")
df

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [57]:
# karakter sütunlarını sayısal verilere çevrilmeli

from sklearn import preprocessing

outlook = df.iloc[0:,0:1].values

# öncelikle outlook'a random değerler
le = preprocessing.LabelEncoder()
outlook[:,0] = le.fit_transform(df.iloc[:,0])
outlook

array([[2],
       [2],
       [0],
       [1],
       [1],
       [1],
       [0],
       [2],
       [2],
       [1],
       [2],
       [0],
       [0],
       [1]], dtype=object)

In [58]:
# bu değerleri sütunlara çevir
ohe = preprocessing.OneHotEncoder()
outlook = ohe.fit_transform(outlook).toarray()
outlookCol = pd.DataFrame(data=outlook, index=range(len(df)), columns=["overcast","rainy","sunny"])
outlookCol

,overcast,rainy,sunny
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
5,0.0,1.0,0.0
6,1.0,0.0,0.0
7,0.0,0.0,1.0
8,0.0,0.0,1.0
9,0.0,1.0,0.0


In [59]:
# aynısını windy ve play için yap
# ama tek sütun almak yeterli, çünkü 2 koşullu durum

playCol = df.iloc[:,-1:].values

le = preprocessing.LabelEncoder()

playCol[:,-1] = le.fit_transform(df.iloc[:,-1])

playCol

array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]], dtype=object)

In [48]:
# ama bunlar yerine bi fonksiyon kullanarak istediğimiz sütun için encoding yapabiliriz

windyPlayCol = df[["windy", "play"]].apply(preprocessing.LabelEncoder().fit_transform)
windyPlayCol

,windy,play
0,0,0
1,1,0
2,0,1
3,0,1
4,0,1
5,1,0
6,1,1
7,0,0
8,0,1
9,0,1


In [86]:
# yeni sütunları birleştirip istenileni ayır

# y
humadity = df.loc[:,"humidity"]
humadityCol = pd.DataFrame(humadity)
humadityCol

,humidity
0,85
1,90
2,86
3,96
4,80
5,70
6,65
7,95
8,70
9,80


In [93]:
# x
newDf = pd.concat([outlookCol, windyPlayCol], axis=1)
newDf = pd.concat([newDf, df.loc[:,"temperature"]], axis=1)
newDf

,overcast,rainy,sunny,windy,play,temperature
0,0.0,0.0,1.0,0,0,85
1,0.0,0.0,1.0,1,0,80
2,1.0,0.0,0.0,0,1,83
3,0.0,1.0,0.0,0,1,70
4,0.0,1.0,0.0,0,1,68
5,0.0,1.0,0.0,1,0,65
6,1.0,0.0,0.0,1,1,64
7,0.0,0.0,1.0,0,0,72
8,0.0,0.0,1.0,0,1,69
9,0.0,1.0,0.0,0,1,75


In [108]:
# train-test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(newDf, humadityCol, test_size=0.33, random_state=0)

In [109]:
# predict

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)

y_predict = regressor.predict(x_test)
print(y_test)
y_predict

    humidity
8         70
6         65
4         80
11        90
2         86


array([[84.45365573],
       [63.93839954],
       [85.76050662],
       [64.21013241],
       [75.06793322]])

In [98]:
# backward ile tahmin

import statsmodels.api as sm

# bütün değişkenleri bir diziye atıp sırasıyla hepsi denenir
# p-value değerleri yani sistemi bozanları sırasıyla çıkarılır

# Beta0 değerleri için hepsine 1 ekliyorum
X = np.append(arr = np.ones((len(df),1)).astype(int), values=newDf, axis=1)

XList = newDf.iloc[:,[0,1,2,3,4,5]].values
XList = np.array(XList, dtype=float)
model = sm.OLS(humadityCol,XList).fit()
print(model.summary())
     

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Thu, 29 Dec 2022   Prob (F-statistic):              0.661
Time:                        13:14:54   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            52.3891     50.214      1.043      0.3

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [101]:
# p-value x4'te en büyük onu çıkartırız. (windy)

XList = newDf.iloc[:,[0,1,2,4,5]].values
XList = np.array(XList, dtype=float)
model = sm.OLS(humadityCol,XList).fit()
print(model.summary())
XList

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Thu, 29 Dec 2022   Prob (F-statistic):              0.546
Time:                        13:19:12   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            36.6564     39.905      0.919      0.3

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


array([[ 0.,  0.,  1.,  0., 85.],
       [ 0.,  0.,  1.,  0., 80.],
       [ 1.,  0.,  0.,  1., 83.],
       [ 0.,  1.,  0.,  1., 70.],
       [ 0.,  1.,  0.,  1., 68.],
       [ 0.,  1.,  0.,  0., 65.],
       [ 1.,  0.,  0.,  1., 64.],
       [ 0.,  0.,  1.,  0., 72.],
       [ 0.,  0.,  1.,  1., 69.],
       [ 0.,  1.,  0.,  1., 75.],
       [ 0.,  0.,  1.,  1., 75.],
       [ 1.,  0.,  0.,  1., 72.],
       [ 1.,  0.,  0.,  1., 81.],
       [ 0.,  1.,  0.,  0., 71.]])

In [112]:
 # attıktan sonra yeniden öğrenme

x_train, x_test, y_train, y_test = train_test_split(XList, humadityCol, test_size=0.33, random_state=0)
regressor.fit(x_train,y_train)

y_predict2 = regressor.predict(x_test)
print(y_test)
y_predict2

    humidity
8         70
6         65
4         80
11        90
2         86


array([[77.98135142],
       [68.29304916],
       [81.05037539],
       [71.44926132],
       [75.78905304]])